# 内容说明
本文件是用于读取已训练的模型，并在测试集得到最终排名预测文件。


# 训练集说明

In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data
!ls /home/aistudio/data/data134077

data134077
CVPR_2022_NAS_Track1_test.json	CVPR_2022_NAS_Track2_train.json
CVPR_2022_NAS_Track2_test.json	resnet48.pdparams


In [2]:
# 读取训练数据, 训练集包含500个模型结构，以及这些结构在cplfw，market1501，dukemtmc等8个任务上的性能排序
import json
with open('/home/aistudio/data/data134077/CVPR_2022_NAS_Track2_train.json', 'r') as f:
    train_data = json.load(f)
print(train_data['arch1'])
print('train_num:',len(train_data.keys()))

{'cplfw_rank': 9, 'market1501_rank': 361, 'dukemtmc_rank': 426, 'msmt17_rank': 433, 'veri_rank': 327, 'vehicleid_rank': 480, 'veriwild_rank': 425, 'sop_rank': 367, 'arch': 'l231131331121121331111211121331321321'}
train_num: 500


# 处理数据


In [3]:
def convert_X(arch_str):
    temp_arch = []
    for elm in arch_str:
        if elm == 'l':
            temp_arch.append(1)
        elif elm == 'j':
            temp_arch.append(2)
        elif elm == 'k':
            temp_arch.append(3)
        else: 
            temp_arch.append(int(elm))
    return(temp_arch)
train_list = [[],[],[],[],[],[],[],[]]
arch_list_train = []
name_list = ['cplfw_rank', 'market1501_rank', 'dukemtmc_rank', 'msmt17_rank', 'veri_rank', 'vehicleid_rank', 'veriwild_rank', 'sop_rank']
for key in train_data.keys():
    for idx, name in enumerate(name_list):
        train_list[idx].append(train_data[key][name])
    arch_list_train.append(convert_X(train_data[key]['arch']))
print(arch_list_train[0])

[1, 2, 3, 1, 1, 3, 1, 3, 3, 1, 1, 2, 1, 1, 2, 1, 3, 3, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 3, 3, 1, 3, 2, 1, 3, 2, 1]


In [4]:
# 安装paddleslim
!pip install paddleslim

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.


In [5]:
# 加载python包
import os
import numpy as np
from paddle.io import Dataset,DataLoader
from paddle.vision import transforms as T
from paddle.nn import functional as F
import cv2
import paddle
import matplotlib.pyplot as plt
import paddle.nn as nn
from tqdm import tqdm
import math
from math import sqrt
import time

import pdb

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


In [6]:
# 记录并计算平均值
class AverageMeter(object):
    """
    Computes and stores the average and current value
    Copied from: https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [7]:
# 模糊化层
# 基于模糊集理论，对于每个输入，通过若干个隶属函数，转化为隶属度（0~1）
class Fuzzify(nn.Layer):
    def __init__(self,dim=2,flag_fix=False): # dim为每个输入对应的隶属函数数目，隶属函数选用高斯函数
        super(Fuzzify, self).__init__()
        self.dim = dim
        if not flag_fix: # 隶属函数的参数是否是固定的
            param_c = self.create_parameter([1,dim]) # 高斯函数的期望值
            self.add_parameter("param_c", param_c)
            param_d = self.create_parameter([1,dim]) # 高斯函数的标准差
            self.add_parameter("param_d", param_d)
            # init
            self.param_c.set_value(np.random.rand(1,dim).astype("float32")*3) # 初始化[0,3)，因为每个输入值取值为{0,1,2,3}
            self.param_d.set_value(np.ones((1,dim)).astype("float32")) # 初始化
        else:
            self.param_c = paddle.zeros([1, dim])
            #self.param_d = paddle.ones([1, dim])
            param_d = self.create_parameter([1,dim])
            self.add_parameter("param_d", param_d)
            # init
            for _ in range(dim):
                self.param_c[0, _] = _
            self.param_d.set_value(np.ones((1,dim)).astype("float32"))

    def forward(self,x):
        x_out = []
        for i in range(self.dim):
            x_out.append(paddle.exp(-(x - self.param_c[0, i]) ** 2 * (2 * self.param_d[0, i] ** 2))) # 除以2\delta^2 改为 乘以2\delta^2
        return paddle.concat(x_out, 1)

In [8]:
class FRNN(nn.Layer): # 模糊粗糙神经网络
    def __init__(self,num_feat,num_mf=2,num_fr=7,num_rgh=3,num_classes=1):
        super(FRNN, self).__init__()
        '''
        FRNN: fuzzification, fuzzy rule, rough, output
        '''

        '''
        self.fuzz = Fuzzify(num_mf,flag_fix=False)
        self.cnsq = nn.Sequential(
            nn.Linear(num_feat,num_feat*num_mf),
            nn.BatchNorm1D(num_feat*num_mf),
            nn.ReLU(),
        )
        self.out  = nn.Sequential(
            nn.Linear(2*num_feat*num_mf,num_classes),
            nn.Sigmoid()
        )
        '''
        self.frnn = nn.Sequential(
            Fuzzify(num_mf,flag_fix=False), # 模糊化层
            #nn.Linear(num_feat*num_mf,num_fr), # 模糊规则层
            #nn.BatchNorm1D(num_fr),
            #nn.ReLU(),
            #nn.Linear(num_fr,num_rgh), # 粗糙层
            #nn.BatchNorm1D(num_rgh),
            #nn.ReLU(),
            #nn.Linear(num_rgh,num_classes), # 输出层
            #nn.Linear(num_fr,num_classes), # 输出层
            nn.Linear(num_feat*num_mf,num_classes), # 输出层
            nn.Sigmoid() # 转到0~1
        )
        self.flag = False

    def forward(self,x):
        '''
        x1 = self.fuzz(x)
        x2 = self.cnsq(x)
        x  = self.out(paddle.concat(x=[x1, x2], axis=1))
        if not self.flag:
            print(x.shape, x)
            self.flag = True
        return x
        '''
        return self.frnn(x)

In [9]:
# 评估网络模型
def evaluation(model, params_file_path, valid_loader,flag_test=False):
    if params_file_path is not None:
        model_state_dict = paddle.load(params_file_path)
        model.load_dict(model_state_dict)
    model.eval()

    all_pd = []
    all_gt = []
    for idx, data in enumerate(valid_loader()):
        if flag_test:
            arch_v = data
        else:
            arch_v, score_label = data

        scores_predict = model(arch_v)

        all_pd.append(scores_predict)
        if not flag_test:
            all_gt.append(score_label)

    all_pd = paddle.concat(all_pd)
    if flag_test: # 测试集没groundtruth
        return all_pd.numpy()
    else:
        all_gt = paddle.concat(all_gt)
        return scipy.stats.stats.kendalltau(all_pd.numpy(), all_gt.numpy())

In [10]:
# 构建数据集
class ArchDataset(Dataset):
    def __init__(self, data, label, transform=None):
        super(ArchDataset, self).__init__()
        self.data = data
        self.label = label
        self.transform = transform

    def __getitem__(self, index):
        if self.label is not None:
            return np.array(self.data[index]).astype("float32"), np.array(self.label[index]).astype("float32")
        else:
            return np.array(self.data[index]).astype("float32")

    def __len__(self):
        return len(self.data)

# 查看测试集

In [11]:
import json
with open('/home/aistudio/data/data134077/CVPR_2022_NAS_Track2_test.json', 'r') as f:
    test_data = json.load(f)
test_data['arch99997']

{'cplfw_rank': 0,
 'market1501_rank': 0,
 'dukemtmc_rank': 0,
 'msmt17_rank': 0,
 'veri_rank': 0,
 'vehicleid_rank': 0,
 'veriwild_rank': 0,
 'sop_rank': 0,
 'arch': 'j121221121221221311331321121221000000'}

# 处理测试集数据

In [12]:
test_arch_list = []
for key in test_data.keys():
    test_arch =  convert_X(test_data[key]['arch'])
    test_arch_list.append(test_arch)
print(test_arch_list[99499])

[1, 2, 2, 1, 3, 3, 1, 1, 1, 1, 2, 2, 1, 1, 3, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 3, 1, 3, 1, 1, 3, 2, 1, 3, 3, 1]


 # 生成提交结果

In [22]:
from paddle.static import InputSpec

frnn_list = []

'''
for i in range(len(train_list[:])):
    # 每个任务有该任务专属的FRNN预测器
    frnn_list.append(FRNN(num_feat=37,num_mf=16,num_fr=8,num_rgh=4,num_classes=1))

for i in range(len(train_list[:])):
    params_file_path = './checkpoint/task' + str(i) + '_best' + '.pdparams'
    model_state_dict = paddle.load(params_file_path)
    frnn_list[i].load_dict(model_state_dict)
    paddle.jit.save(
        layer=frnn_list[i], 
        path='./save/task' + str(i) + '_best' + '.pdmodel',
        input_spec=[InputSpec(shape=[None, 37], dtype='float32')]
        )
'''

for i in range(len(train_list[:])):
    frnn_list.append(paddle.jit.load('./save/task' + str(i) + '_best' + '.pdmodel'))

##
batch_size = 512
rank_all = []
pd_all_frnn = [] # 预测值
rank_all_frnn = [] # 预测排序
for task in range(len(name_list)):
    print('Predict the rank of:', name_list[task])
    
    X_test_k = np.array(test_arch_list)
    test_dataset = ArchDataset(X_test_k, None)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
    # 测试最优模型参数
    all_pd = evaluation(frnn_list[task], None,
                        test_dataloader, flag_test=True)
    pd_all_frnn.append(all_pd)
    tmp = np.argsort(all_pd.reshape(-1))
    tmp2 = np.argsort(tmp) # 获得排名
    rank_all_frnn.append(tmp2)

Predict the rank of: cplfw_rank
Predict the rank of: market1501_rank
Predict the rank of: dukemtmc_rank
Predict the rank of: msmt17_rank
Predict the rank of: veri_rank
Predict the rank of: vehicleid_rank
Predict the rank of: veriwild_rank
Predict the rank of: sop_rank


In [23]:
for idx,key in enumerate(test_data.keys()):
    test_data[key]['cplfw_rank'] = int(rank_all_frnn[0][idx])
    test_data[key]['market1501_rank'] = int(rank_all_frnn[1][idx])
    test_data[key]['dukemtmc_rank'] = int(rank_all_frnn[2][idx])
    test_data[key]['msmt17_rank'] = int(rank_all_frnn[3][idx])
    test_data[key]['veri_rank'] = int(rank_all_frnn[4][idx])
    test_data[key]['vehicleid_rank'] = int(rank_all_frnn[5][idx])
    test_data[key]['veriwild_rank'] = int(rank_all_frnn[6][idx])
    test_data[key]['sop_rank'] = int(rank_all_frnn[7][idx])
print('Ready to save results!')
with open('./CVPR_2022_NAS_Track2_submit_A_frnn.json', 'w') as f: # 得到提出方案的预测值
    json.dump(test_data, f)


Ready to save results!


In [ ]:
!zip 

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 